Read the Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Veriyi yükle
df = pd.read_excel("kartData1.xlsx")

In [2]:

df["trxdate"] = pd.to_datetime(df["trxdate"], format ="%d/%m/%Y")

df["trx_month"] = df["trxdate"].dt.to_period('M')
start_period = pd.Period("2023-07")
end_period = pd.Period("2024-04")

df_selected_period = df[(df["trx_month"] >= start_period) & (df["trx_month"] <= end_period)]

df_selected_period["TotalAmountLC"] = pd.to_numeric(df_selected_period["TotalAmountLC"], errors='coerce')

eight_months_TotalAmountLC = df_selected_period.groupby(["Customer_id"])["TotalAmountLC"].sum().reset_index()

sorted_eight_months_TotalAmountLC = eight_months_TotalAmountLC["TotalAmountLC"].sort_values()

customers = range(1, len(sorted_eight_months_TotalAmountLC) + 1)
eigtht_months_TotalAmountLC_values = sorted_eight_months_TotalAmountLC.values

total_TotalAmountLC = eight_months_TotalAmountLC["TotalAmountLC"].sum()

percentile_80 = sorted_eight_months_TotalAmountLC.quantile(0.8)

In [3]:

monthly_TotalAmountLC_per_customer = df_selected_period.groupby(["Customer_id", "trx_month"])["TotalAmountLC"].sum().reset_index()

all_customers = df_selected_period["Customer_id"].unique()
all_months = pd.period_range(start=start_period, end=end_period, freq='M')

customer_month_combinations = pd.MultiIndex.from_product([all_customers, all_months], names=['Customer_id', 'trx_month'])
full_customer_month_TotalAmountLC = pd.DataFrame(index=customer_month_combinations).reset_index()

full_customer_month_TotalAmountLC = pd.merge(full_customer_month_TotalAmountLC, monthly_TotalAmountLC_per_customer, on=["Customer_id", "trx_month"], how="left")

# Filling the TotalAmountLC values with 0 when the customer had no record for that month
full_customer_month_TotalAmountLC["TotalAmountLC"].fillna(0, inplace=True)


In [4]:
sorted_TotalAmountLC = df_selected_period.groupby("Customer_id")["TotalAmountLC"].sum().sort_values()

customers = range(1, len(sorted_TotalAmountLC) + 1)
monthly_TotalAmountLC_values = sorted_TotalAmountLC.values

Calculation of the slope between months i-1 and i-2

In [5]:

slopes = []
for i in range(len(sorted_eight_months_TotalAmountLC) - 1):
    slope = (sorted_eight_months_TotalAmountLC.iloc[i-1] - sorted_eight_months_TotalAmountLC.iloc[i-2])
    slopes.append(slope)

print(slopes)


Finding the TotalAmountLC values for a specific customer, used for checking if everything is alright


In [6]:
customer_id = 1108

customer_monthly_TotalAmountLC = full_customer_month_TotalAmountLC[full_customer_month_TotalAmountLC["Customer_id"] == customer_id]

print("Müşteri ID:", customer_id)
for index, row in customer_monthly_TotalAmountLC.iterrows():
    print("Aylık TotalAmountLC ({}) : {:.2f} TL".format(row["trx_month"], row["TotalAmountLC"]))


months = customer_monthly_TotalAmountLC["trx_month"]
TotalAmountLC_values = customer_monthly_TotalAmountLC["TotalAmountLC"]


months = months.dt.to_timestamp()

plt.figure(figsize=(10, 6))
plt.plot(months, TotalAmountLC_values, marker='o', color='blue')
plt.title('Customer Expenditure Trend'.format(customer_id))
plt.xlabel('Month')
plt.ylabel('Amount of expenditure')
plt.grid(True)
plt.show()
print(customer_monthly_TotalAmountLC)


Müşteri ID: 1108


#### Remove Outlier from the Data

In [7]:
mean_TotalAmountLC = customer_monthly_TotalAmountLC["TotalAmountLC"].mean()
std_TotalAmountLC = customer_monthly_TotalAmountLC["TotalAmountLC"].std()

threshold = 2 

outliers = customer_monthly_TotalAmountLC[
    (customer_monthly_TotalAmountLC["TotalAmountLC"] < mean_TotalAmountLC - threshold * std_TotalAmountLC) |
    (customer_monthly_TotalAmountLC["TotalAmountLC"] > mean_TotalAmountLC + threshold * std_TotalAmountLC)
]

non_outlier_data = customer_monthly_TotalAmountLC[~customer_monthly_TotalAmountLC.index.isin(outliers.index)]
non_outlier_mean = non_outlier_data["TotalAmountLC"].mean()

customer_monthly_TotalAmountLC.loc[outliers.index, "TotalAmountLC"] = non_outlier_mean

print("Müşteri ID:", customer_id)
for index, row in customer_monthly_TotalAmountLC.iterrows():
    print("Aylık TotalAmountLC ({}) : {:.2f} TL".format(row["trx_month"], row["TotalAmountLC"]))

months_filtered = customer_monthly_TotalAmountLC["trx_month"]
TotalAmountLC_values_filtered = customer_monthly_TotalAmountLC["TotalAmountLC"]
months_filtered = months_filtered.dt.to_timestamp()

plt.figure(figsize=(10, 6))
plt.plot(months_filtered, TotalAmountLC_values_filtered, marker='o', color='green')
plt.title('The Expenditure Trend of the Customer {} After Replacing Outliers with Non-Outlier Mean'.format(customer_id))
plt.xlabel('Month')
plt.ylabel('Amount of Expenditure')
plt.grid(True)
plt.show()


Müşteri ID: 1108


#### Remove the Outliers from All Customers

In [8]:
import pandas as pd
import numpy as np

outlierRemoved_data = []

for customer_id, group in full_customer_month_TotalAmountLC.groupby('Customer_id'):
    mean_TotalAmountLC = group["TotalAmountLC"].mean()
    
    mean_TotalAmountLC = group["TotalAmountLC"].mean()
    std_TotalAmountLC = group["TotalAmountLC"].std()
   
    outliers = group[(group["TotalAmountLC"] < mean_TotalAmountLC - threshold * std_TotalAmountLC) |
                     (group["TotalAmountLC"] > mean_TotalAmountLC + threshold * std_TotalAmountLC)]
    
    # Change the outlier values with the average of the data points
    group.loc[outliers.index, "TotalAmountLC"] = mean_TotalAmountLC
    
    outlierRemoved_data.append(group)

outlierRemoved_customer_monthly_TotalAmountLC = pd.concat(outlierRemoved_data)

print(outlierRemoved_customer_monthly_TotalAmountLC)


        Customer_id trx_month  TotalAmountLC
643720     16082019   2023-07         119.99
643721     16082019   2023-08           0.00
643722     16082019   2023-09           0.00
643723     16082019   2023-10           0.00
643724     16082019   2023-11           0.00
...             ...       ...            ...
5          97425780   2023-12         710.00
6          97425780   2024-01         119.99
7          97425780   2024-02         468.00
8          97425780   2024-03         340.00
9          97425780   2024-04           0.00

[643730 rows x 3 columns]


#### Calculate the Average Negative Slope

In [9]:
def calculate_slope(TotalAmountLC_values):
    slopes_percent = []
    for i in range(1, len(TotalAmountLC_values)):
        slope_percent = ((TotalAmountLC_values[i-2] - TotalAmountLC_values[i-1]) / TotalAmountLC_values[i-2])  if TotalAmountLC_values[i-2] != 0 else 0
        slopes_percent.append(slope_percent)
    return slopes_percent

customer_monthly_slopes_percent = {}

for customer_id, month_TotalAmountLC in outlierRemoved_customer_monthly_TotalAmountLC.groupby("Customer_id"):
    TotalAmountLC_values_outlierRemoved = month_TotalAmountLC["TotalAmountLC"].values
    slopes_percent = calculate_slope(TotalAmountLC_values_outlierRemoved)
    for month, slope_percent in zip(month_TotalAmountLC["trx_month"], slopes_percent):
        key = (customer_id, month)
        customer_monthly_slopes_percent[key] = slope_percent

sorted_customer_monthly_slopes_percent = sorted(customer_monthly_slopes_percent.items(), key=lambda x: x[0][0])

for key, slope_percent in sorted_customer_monthly_slopes_percent:
    customer_id, month = key
    print("Customer ID:", customer_id, "Month:", month+1, "Slope (%):", slope_percent)

negative_slopes = [slope_percent for slope_percent in customer_monthly_slopes_percent.values() if slope_percent < 0]

average_negative_slope = np.mean(negative_slopes)

print("Average Negative Slope: {:.2f}%".format(average_negative_slope))
slope_2023_07 = customer_monthly_slopes_percent.get((customer_id, "2023-07"), None)
print(slope_2023_07)


#### Saving the Slope Values to a CSV File

In [ ]:
import pandas as pd

slope_df = pd.DataFrame(columns=["Customer_id", "trxdate", "Slope"])

for (customer_id, month), slope_percent in customer_monthly_slopes_percent.items():
    slope_df = pd.concat([slope_df, pd.DataFrame({"Customer_id": [customer_id], "trxdate": [month + 1], "Slope": [slope_percent]})], ignore_index=True)

slope_df["trxdate"] = slope_df["trxdate"].dt.to_timestamp()

slope_df["trxdate"] = slope_df["trxdate"].dt.strftime('%Y-%m')


print(slope_df.head())
slope_df.to_csv("slope.csv", index=False)

  Customer_id  trxdate    Slope
0    16082019  2023-08  0.63506
1    16082019  2023-09  1.00000
2    16082019  2023-10  0.00000
3    16082019  2023-11  0.00000
4    16082019  2023-12  0.00000


#### Saving the Moving Average Values to a CSV File

In [ ]:
import pandas as pd

MA_df = pd.DataFrame(columns=["Customer_id", "trxdate", "Moving_Average"])

for Customer_id, group in outlierRemoved_customer_monthly_TotalAmountLC.groupby("Customer_id"):
    TotalAmountLC_values = group["TotalAmountLC"].values
    slopes_percent = calculate_slope(TotalAmountLC_values)
    for i, (month, TotalAmountLC, slope_percent) in enumerate(zip(group["trx_month"]+1, TotalAmountLC_values, slopes_percent)):
        moving_average = group["TotalAmountLC"].iloc[:i].mean()
        moving_average = 0 if pd.isnull(moving_average) else moving_average
        MA_df = pd.concat([MA_df, pd.DataFrame({"Customer_id": [Customer_id], "trxdate": [month], "Moving_Average": [moving_average]})], ignore_index=True)
print(MA_df.head())
MA_df.to_csv("MA.csv")


#### Calculating Churn for Every Row & Saving the Churn Values to a CSV File

In [ ]:
import pandas as pd

churn_data = []

for customer_id, group in outlierRemoved_customer_monthly_TotalAmountLC.groupby("Customer_id"):
    TotalAmountLC_values = group["TotalAmountLC"].values
    slopes_percent = calculate_slope(TotalAmountLC_values)  

    churn_data.append((customer_id, "2023-07", 0))

    for i, (month, TotalAmountLC, slope_percent) in enumerate(zip(group["trx_month"]+1, TotalAmountLC_values, slopes_percent)):
        if month != "2023-07":
            if slope_percent < 0 and TotalAmountLC < group["TotalAmountLC"].iloc[:i].mean():
                churn_data.append((customer_id, month, 1))  # Churn
            elif TotalAmountLC == 0:  # ELabel Customer as Churn if She made no transaction in that month
                churn_data.append((customer_id, month, 1))  # Churn
            else:
                churn_data.append((customer_id, month, 0))  # Nonchurn

churn_df = pd.DataFrame(churn_data, columns=["Customer_id", "trxdate", "Churn"])
churn_df.to_csv("churn_analysis.csv", index=False)

churn_count = churn_df["Churn"].sum()

total_months = len(churn_df)

churn_rate = churn_count / total_months

print(churn_df)
print("Toplam Ay Sayısı:", total_months)
print("Churn Olma Sayısı:", churn_count)
print("Churn Gerçekleşme Oranı: {:.2%}".format(churn_rate))


        Customer_id  trxdate  Churn
0          16082019  2023-07      0
1          16082019  2023-08      0
2          16082019  2023-09      1
3          16082019  2023-10      1
4          16082019  2023-11      1
...             ...      ...    ...
643725     97425780  2023-12      1
643726     97425780  2024-01      0
643727     97425780  2024-02      0
643728     97425780  2024-03      0
643729     97425780  2024-04      0

[643730 rows x 3 columns]
Toplam Ay Sayısı: 643730
Churn Olma Sayısı: 163957
Churn Gerçekleşme Oranı: 25.47%
